<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-10 15:06:35
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.41 C
-------------------
Today PnL: 69.46 K (0.49%)
Current PnL: -28.16 L (-17.96%)
CY Booked + Current PnL: -13.42 L (-8.56%)
-------------------
Total profit:  1.71 L
Total loss:  -29.86 L
-------------------
Total Booked + Current PnL: 13.56 L (10.45%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.43%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.54 L (66.18%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.95,2.62,18.47,21.57,18571.0,2565.0,100545.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.73,XR,ATH,FINANCE
18,COALINDIA,0.02,6.18,12.28,19.23,19619.0,9306.0,159766.0,484.83,24.27,54.0,L-LC,12.33,182.0,0.47,1.13,28.78,XY25,ATH,MINING
77,TTKPRESTIG,0.79,-24.28,32.19,0.09,24564.0,-24469.0,76308.0,770.00,77.16,48.0,M-SC,8.41,253.0,-1.00,0.54,5.02,OX40N,NTT,DURABLES
5,ANGELONE,2.36,13.82,9.12,24.20,31687.0,42200.0,347450.0,3033.00,61.96,62.0,X-SC,2.40,99.0,1.33,2.46,45.52,X40N,NTT,FINANCE
36,ICICIGI,1.31,1.33,17.77,19.34,36034.0,2669.0,202778.0,2252.93,-20.03,58.0,X-MC,3.64,71.0,0.07,1.44,17.48,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JCHAC,7.65,-37.76,60.65,-0.01,48246.0,-48257.0,79548.0,2282.00,16182.61,54.0,M-SC,17.08,235.0,-1.00,0.56,10.29,OX40N,BTT,AC
7,ATULAUTO,6.78,-10.67,62.95,45.57,110857.0,-21029.0,176103.0,844.00,3818.92,77.0,M-SC,7.87,245.0,-0.19,1.25,33.27,XY24,NTT,AUTO
14,BSOFT,4.57,-15.60,78.65,50.79,93369.0,-21938.0,118715.0,831.70,8.23,62.0,H-SC,7.30,171.0,-0.23,0.84,38.22,XR,ATH,IT
55,RAJESHEXPO,4.24,-63.51,174.22,0.07,87211.0,-87119.0,50058.0,518.00,1734.60,61.0,L-SC,10.38,269.0,-1.00,0.35,24.50,OX40N,NTT,JEWELLERY
26,FINCABLES,3.95,4.36,99.23,107.91,152893.0,6435.0,154079.0,1641.55,-15.28,70.0,M-SC,10.90,220.0,0.04,1.09,15.92,OX40N,ATH,CABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,-3.90,-6.42,44.94,35.63,88134.0,-13459.0,196115.0,1286.17,-19.40,46.0,H-MC,6.16,119.0,-0.15,1.39,9.18,AR,ATH,PHARMA
79,VAIBHAVGBL,-3.86,-23.16,109.24,60.77,153412.0,-42339.0,140436.0,521.00,61.98,59.0,H-SC,3.12,158.0,-0.28,0.99,27.32,XR,NTT,JEWELLERY
76,TRIDENT,-2.93,-26.37,74.55,28.51,50638.0,-24330.0,67925.0,48.00,-12.81,54.0,M-SC,9.13,226.0,-0.48,0.48,16.23,XR,NTT,TEXTILES
83,WHIRLPOOL,-2.53,-29.16,156.79,81.92,127514.0,-33470.0,81328.0,2270.00,-58.54,60.0,M-SC,32.30,218.0,-0.26,0.58,14.24,XR,NTT,DURABLES
51,MEDANTA,-1.94,-4.95,30.53,24.08,36841.0,-6280.0,120670.0,1486.00,-12.71,52.0,X-SC,11.48,91.0,-0.17,0.85,10.52,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,3.95,4.36,99.23,107.91,152893.0,6435.0,154079.0,1641.55,-15.28,70.0,M-SC,10.90,220.0,0.04,1.09,15.92,OX40N,ATH,CABLES
66,SIS,1.17,-19.70,51.01,21.26,45481.0,-21871.0,89161.0,528.00,2109.33,68.0,H-SC,4.37,163.0,-0.48,0.63,22.92,OX40N,NTT,MISC
72,TATAELXSI,3.00,-22.32,69.96,32.03,71646.0,-29425.0,102410.0,9161.00,-15.87,50.0,H-SC,6.20,157.0,-0.41,0.72,14.71,OX40N,NTT,IT
77,TTKPRESTIG,0.79,-24.28,32.19,0.09,24564.0,-24469.0,76308.0,770.00,77.16,48.0,M-SC,8.41,253.0,-1.00,0.54,5.02,OX40N,NTT,DURABLES
40,INDIGOPNTS,-0.78,-26.71,36.47,0.02,46656.0,-46628.0,127931.0,1408.00,95.31,36.0,M-SC,6.13,240.0,-1.00,0.91,11.28,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.02,6.18,12.28,19.23,19619.0,9306.0,159766.0,484.83,24.27,54.0,L-LC,12.33,182.0,0.47,1.13,28.78,XY25,ATH,MINING
50,MASFIN,0.95,2.62,18.47,21.57,18571.0,2565.0,100545.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.73,XR,ATH,FINANCE
26,FINCABLES,3.95,4.36,99.23,107.91,152893.0,6435.0,154079.0,1641.55,-15.28,70.0,M-SC,10.90,220.0,0.04,1.09,15.92,OX40N,ATH,CABLES
1,ABB,1.13,7.91,34.90,45.57,98527.0,20693.0,282312.0,7934.00,-30.61,74.0,H-MC,3.59,121.0,0.21,2.00,25.37,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.13,7.91,34.90,45.57,98527.0,20693.0,282312.0,7934.00,-30.61,74.0,H-MC,3.59,121.0,0.21,2.00,25.37,AR,NTT,ELECTRICAL
50,MASFIN,0.95,2.62,18.47,21.57,18571.0,2565.0,100545.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.73,XR,ATH,FINANCE
26,FINCABLES,3.95,4.36,99.23,107.91,152893.0,6435.0,154079.0,1641.55,-15.28,70.0,M-SC,10.90,220.0,0.04,1.09,15.92,OX40N,ATH,CABLES
39,INDIAMART,-0.22,-3.60,114.43,106.72,136057.0,-4436.0,118900.0,4810.62,-52.34,53.0,H-SC,7.92,138.0,-0.03,0.84,18.94,AR,ATH,MISC
84,WIPRO,0.72,-4.95,81.24,72.27,130844.0,-8389.0,161059.0,420.00,-18.33,38.0,M-LC,3.97,101.0,-0.06,1.14,3.52,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.87,-18.34,48.34,21.14,89744.0,-41687.0,185652.0,680.00,-24.04,26.0,X-MC,9.47,75.0,-0.46,1.31,0.00,XY24,NTT,PAINTS
6,ASIANPAINT,-1.16,-16.49,44.82,20.95,94238.0,-41510.0,210258.0,3460.25,-17.36,28.0,X-LC,8.44,36.0,-0.44,1.49,13.27,X40,ATH,PAINTS
42,INFY,0.09,2.01,31.60,34.24,100861.0,6292.0,319180.0,1972.00,-20.67,30.0,X-LC,5.34,6.0,0.06,2.26,10.46,X40,NTT,IT
8,AWL,-0.58,-31.93,130.93,57.19,290653.0,-104136.0,221991.0,485.00,-67.84,34.0,X-MC,16.99,58.0,-0.36,1.57,1.17,XY24,NTT,FMCG
33,HCLTECH,-1.67,2.22,21.14,23.83,52280.0,5382.0,247306.0,1908.19,6.82,34.0,X-LC,6.85,13.0,0.10,1.75,18.65,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,1.30,-17.88,44.36,18.55,137785.0,-67631.0,310606.0,4311.59,-29.13,39.0,X-LC,1.19,3.0,-0.49,2.20,5.67,X40,ATH,IT
81,VBL,-0.02,-7.64,46.89,35.67,136790.0,-24116.0,291726.0,671.64,-18.58,42.0,X-LC,1.27,5.0,-0.18,2.06,5.16,X40N,ATH,FMCG
21,DABUR,1.15,1.72,41.26,43.70,104118.0,4277.0,252345.0,735.00,-6.68,59.0,X-MC,1.96,73.0,0.04,1.79,17.03,XY24,BTT,FMCG
5,ANGELONE,2.36,13.82,9.12,24.20,31687.0,42200.0,347450.0,3033.00,61.96,62.0,X-SC,2.40,99.0,1.33,2.46,45.52,X40N,NTT,FINANCE
34,HINDUNILVR,0.90,-4.42,18.93,13.67,46509.0,-11373.0,245690.0,2922.00,-11.61,63.0,X-LC,2.47,9.0,-0.24,1.74,15.99,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.87,-18.34,48.34,21.14,89744.0,-41687.0,185652.0,680.00,-24.04,26.0,X-MC,9.47,75.0,-0.46,1.31,0.00,XY24,NTT,PAINTS
8,AWL,-0.58,-31.93,130.93,57.19,290653.0,-104136.0,221991.0,485.00,-67.84,34.0,X-MC,16.99,58.0,-0.36,1.57,1.17,XY24,NTT,FMCG
2,ABBOTINDIA,-0.66,-8.42,28.92,18.07,47371.0,-15055.0,163800.0,35195.00,-21.66,45.0,X-MC,9.63,64.0,-0.32,1.16,2.50,X40,ATH,PHARMA
53,PGHH,0.47,-10.33,47.21,32.01,90394.0,-22048.0,191472.0,17616.87,-36.94,47.0,X-MC,8.91,60.0,-0.24,1.36,3.49,X40,ATH,FMCG
3,ACC,-0.57,-28.71,130.27,64.15,220977.0,-68321.0,169630.0,3906.00,-58.50,48.0,X-SC,7.65,84.0,-0.31,1.20,3.69,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,2.84,-26.87,95.33,42.84,45318.0,-17468.0,47538.0,424.00,-52.07,63.0,X-SC,20.34,83.0,-0.39,0.34,14.35,XY24,NTT,MISC
58,RELAXO,0.53,-51.69,215.28,52.31,150963.0,-75036.0,70124.0,1176.00,-48.60,43.0,X-SC,13.06,92.0,-0.50,0.50,4.28,X40N,NTT,FOOTWEAR
12,BATAINDIA,3.73,-40.84,128.53,35.20,97841.0,-52547.0,76123.0,2096.00,-0.22,60.0,X-SC,21.06,93.0,-0.54,0.54,8.18,X40,NTT,FOOTWEAR
51,MEDANTA,-1.94,-4.95,30.53,24.08,36841.0,-6280.0,120670.0,1486.00,-12.71,52.0,X-SC,11.48,91.0,-0.17,0.85,10.52,XY24,NTT,HEALTHCARE
35,HONAUT,-1.08,-21.07,81.57,43.32,104866.0,-34312.0,128560.0,58357.33,-32.13,43.0,X-SC,9.76,90.0,-0.33,0.91,4.38,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.43,-21.23,40.63,10.78,76915.0,-51013.0,189305.0,452.00,-53.00,42.0,X-LC,18.73,1.0,-0.66,1.34,4.27,X40,NTT,FMCG
75,TMPV,0.68,-22.13,57.92,22.98,101891.0,-49981.0,175917.0,600.00,-19.41,66.0,X-LC,3.91,2.0,-0.49,1.24,12.46,XY24,NTT,AUTO
73,TCS,1.30,-17.88,44.36,18.55,137785.0,-67631.0,310606.0,4311.59,-29.13,39.0,X-LC,1.19,3.0,-0.49,2.20,5.67,X40,ATH,IT
81,VBL,-0.02,-7.64,46.89,35.67,136790.0,-24116.0,291726.0,671.64,-18.58,42.0,X-LC,1.27,5.0,-0.18,2.06,5.16,X40N,ATH,FMCG
42,INFY,0.09,2.01,31.60,34.24,100861.0,6292.0,319180.0,1972.00,-20.67,30.0,X-LC,5.34,6.0,0.06,2.26,10.46,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.03,-31.47,93.92,32.89,49690.0,-24300.0,52907.0,1800.00,-753.40,60.0,L-MC,11.49,259.0,-0.49,0.37,45.70,XR,NTT,BANKS
67,SONACOMS,0.34,-7.90,51.02,39.09,47535.0,-7989.0,93170.0,804.02,-27.96,68.0,M-SC,2.32,227.0,-0.17,0.66,31.81,AR,ATH,AUTO
50,MASFIN,0.95,2.62,18.47,21.57,18571.0,2565.0,100545.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.73,XR,ATH,FINANCE
14,BSOFT,4.57,-15.60,78.65,50.79,93369.0,-21938.0,118715.0,831.70,8.23,62.0,H-SC,7.30,171.0,-0.23,0.84,38.22,XR,ATH,IT
79,VAIBHAVGBL,-3.86,-23.16,109.24,60.77,153412.0,-42339.0,140436.0,521.00,61.98,59.0,H-SC,3.12,158.0,-0.28,0.99,27.32,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,2.36,13.82,9.12,24.20,31687.0,42200.0,347450.0,3033.00,61.96,62.0,X-SC,2.40,99.0,1.33,2.46,45.52,X40N,NTT,FINANCE
7,ATULAUTO,6.78,-10.67,62.95,45.57,110857.0,-21029.0,176103.0,844.00,3818.92,77.0,M-SC,7.87,245.0,-0.19,1.25,33.27,XY24,NTT,AUTO
67,SONACOMS,0.34,-7.90,51.02,39.09,47535.0,-7989.0,93170.0,804.02,-27.96,68.0,M-SC,2.32,227.0,-0.17,0.66,31.81,AR,ATH,AUTO
1,ABB,1.13,7.91,34.90,45.57,98527.0,20693.0,282312.0,7934.00,-30.61,74.0,H-MC,3.59,121.0,0.21,2.00,25.37,AR,NTT,ELECTRICAL
11,BANDHANBNK,3.12,-19.60,139.33,92.42,312035.0,-54605.0,223954.0,400.00,57.33,72.0,H-SC,8.79,169.0,-0.17,1.58,29.43,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.34
1,25,44.53
2,50,76.38


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.91
MC    30.64
LC    23.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.65
X40      23.21
X40N     15.19
AR        8.91
XR        8.91
XY25      8.69
OX40N     7.55
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.03
H-SC    23.00
X-LC    20.09
M-SC    11.71
X-SC    10.41
H-MC     4.86
M-MC     1.38
M-LC     1.14
L-LC     1.13
H-LC     1.10
L-SC     0.79
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.97,-12.00,49.64
FINANCE,13.46,-9.81,53.02
IT,12.18,-24.26,86.34
MISC,7.16,-28.49,81.04
ELECTRICAL,6.05,-9.75,49.79
PAINTS,5.06,-29.27,48.48
INSURANCE,4.72,-1.65,36.43
PHARMA,4.03,-3.28,35.79
AUTO,3.15,-17.75,58.47


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3205320.0,21
AR,1313726.0,10
XR,1229889.0,12
X40,1218541.0,15
X40N,977541.0,10
OX40N,723579.0,10
XY25,391953.0,6
SR,293583.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3540727.0,24
X-MC,1635182.0,16
M-SC,1405790.0,15
X-LC,1051866.0,12
X-SC,821623.0,9
H-MC,392997.0,3
L-SC,178258.0,2
M-LC,130844.0,1
H-LC,77897.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219609.0      6
           AR         929137.0      5
M-SC       XY24       789481.0      6
H-SC       XR         780156.0      7
X-MC       X40        536461.0      7
X-LC       X40        519149.0      6
X-MC       XY24       484515.0      3
           X40N       401699.0      4
X-SC       X40N       355556.0      4
H-SC       OX40N      318242.0      4
M-SC       OX40N      318126.0      5
X-SC       XY24       303136.0      3
H-SC       SR         293583.0      2
X-LC       X40N       220286.0      2
X-MC       XY25       212507.0      2
H-MC       XY24       206336.0      1
X-LC       XY24       202243.0      2
H-MC       AR         186661.0      2
M-SC       XR         178152.0      2
X-SC       X40        162931.0      2
M-LC       XR         130844.0      1
M-SC       AR         120031.0      2
X-LC       XY25       110188.0      2
L-SC       XR          91047.0      1
           OX40N       87211.0      1
H-LC       AR          77897.0      1
L-MC       XR          49690.0      1
M-MC       XY25        49639.0      1
L-LC       XY25        19619.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
